In [18]:
import torch
from transformers import GPT2Tokenizer, GPT2TokenizerFast

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [20]:
version = "gpt2"
sequence = "The quick brown fox jumps over the lazy dog."
max_length = 20

# GPT2Tokenizer

In [21]:
tokenizer: GPT2Tokenizer = GPT2Tokenizer.from_pretrained(version)
tokenizer

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

In [22]:
# 需要手动设置pad_token
tokenizer.pad_token = tokenizer.eos_token

## special ids and tokens

In [23]:
print(tokenizer.all_special_ids)
print(tokenizer.all_special_tokens)

[50256, 50256]
['<|endoftext|>', '<|endoftext|>']


## tokenize(sequence) 对文本进行分词

In [24]:
tokens = tokenizer.tokenize(sequence)
tokens

['The',
 'Ġquick',
 'Ġbrown',
 'Ġfox',
 'Ġjumps',
 'Ġover',
 'Ġthe',
 'Ġlazy',
 'Ġdog',
 '.']

## convert_tokens_to_ids 将分词后的token映射为数字

In [25]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[464, 2068, 7586, 21831, 18045, 625, 262, 16931, 3290, 13]

## encode = tokenize + convert_tokens_to_ids

In [26]:
print(tokenizer.encode(sequence, add_special_tokens=False))
ids = tokenizer.encode(sequence, add_special_tokens=True)
ids

[464, 2068, 7586, 21831, 18045, 625, 262, 16931, 3290, 13]


[464, 2068, 7586, 21831, 18045, 625, 262, 16931, 3290, 13]

In [27]:
# 没有开始的token
tokenizer.convert_ids_to_tokens([1])

['"']

## batch_encode_plus = batch encode

In [28]:
tokenizer.batch_encode_plus([sequence], add_special_tokens=True, padding=True)

{'input_ids': [[464, 2068, 7586, 21831, 18045, 625, 262, 16931, 3290, 13]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

## convert_ids_to_tokens 将数字映射为token

In [29]:
print(tokenizer.convert_ids_to_tokens(ids))
tokens = tokenizer.convert_ids_to_tokens(ids, skip_special_tokens=True)
tokens

['The', 'Ġquick', 'Ġbrown', 'Ġfox', 'Ġjumps', 'Ġover', 'Ġthe', 'Ġlazy', 'Ġdog', '.']


['The',
 'Ġquick',
 'Ġbrown',
 'Ġfox',
 'Ġjumps',
 'Ġover',
 'Ġthe',
 'Ġlazy',
 'Ġdog',
 '.']

## convert_tokens_to_string 将token转换为string

In [30]:
tokenizer.convert_tokens_to_string(tokens)

'The quick brown fox jumps over the lazy dog.'

## decode = convert_ids_to_tokens + convert_tokens_to_string

In [31]:
print(tokenizer.decode(ids))
print(tokenizer.decode(ids, skip_special_tokens=True))

The quick brown fox jumps over the lazy dog.
The quick brown fox jumps over the lazy dog.


## batch_decode = batch decode

In [32]:
print(tokenizer.batch_decode([ids]))
print(tokenizer.batch_decode([ids], skip_special_tokens=True))

['The quick brown fox jumps over the lazy dog.']
['The quick brown fox jumps over the lazy dog.']


## tokenizer([sequence])

In [33]:
inputs = tokenizer(
    sequence,                           # 单个句子
    truncation = True,                  # 超出max_length截断处理
    padding = True,                     # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    # max_length = max_length,          # 最长长度,不设置默认为模型最大长度
    add_special_tokens = True,          # text添加特殊key
    return_length = True,               # 返回有效长度
    return_overflowing_tokens = False,  # 返回所有的文本片段（由于文本比较长，默认情况下超过预设截断长度的token会被丢失。如果设置了return_overflowing_tokens=True则会返回所有的token片段）。
    return_tensors = "pt"               # 返回数据格式 np pt tf jax
)

print(inputs.keys())
print(inputs["input_ids"])      # 对应文字id
print(inputs["attention_mask"]) # 对应是否是文字
print(inputs["length"])         # 对应总长度长度

dict_keys(['input_ids', 'attention_mask', 'length'])
tensor([[  464,  2068,  7586, 21831, 18045,   625,   262, 16931,  3290,    13]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([10])


In [34]:
inputs = tokenizer(
    [sequence] * 2,                     # 句子batch
    truncation = True,                  # 超出max_length截断处理
    padding = True,                     # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    # max_length = max_length,          # 最长长度,不设置默认为模型最大长度
    return_length = True,               # 返回有效长度
    return_overflowing_tokens = False,  # 返回所有的文本片段（由于文本比较长，默认情况下超过预设截断长度的token会被丢失。如果设置了return_overflowing_tokens=True则会返回所有的token片段）。
    return_tensors = "pt"               # 返回数据格式 np pt tf jax
).to(device)

print(inputs.keys())
print(inputs["input_ids"])
print(inputs["attention_mask"]) # 对应是否是文字
print(inputs["length"])         # 对应有效文字长度

dict_keys(['input_ids', 'length', 'attention_mask'])
tensor([[  464,  2068,  7586, 21831, 18045,   625,   262, 16931,  3290,    13],
        [  464,  2068,  7586, 21831, 18045,   625,   262, 16931,  3290,    13]],
       device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
tensor([10, 10], device='cuda:0')


# GPT2TokenizerFast

In [38]:
tokenizer: GPT2TokenizerFast = GPT2TokenizerFast.from_pretrained(version)
tokenizer

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)

In [39]:
# 需要手动设置pad_token
tokenizer.pad_token = tokenizer.eos_token

In [40]:
inputs = tokenizer(
    [sequence] * 2,                     # 句子batch
    truncation = True,                  # 超出max_length截断处理
    padding = True,                     # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    # max_length = max_length,          # 最长长度,不设置默认为模型最大长度
    add_special_tokens = True,          # text添加特殊key
    return_length = True,               # 返回有效长度
    return_overflowing_tokens = False,  # 返回所有的文本片段（由于文本比较长，默认情况下超过预设截断长度的token会被丢失。如果设置了return_overflowing_tokens=True则会返回所有的token片段）。
    return_tensors = "pt"               # 返回数据格式 np pt tf jax
).to(device)

print(inputs.keys())
print(inputs["input_ids"])
print(inputs["attention_mask"]) # 对应是否是文字
print(inputs["length"])         # 返回文字长度时最长长度,结果不对

dict_keys(['input_ids', 'attention_mask', 'length'])
tensor([[  464,  2068,  7586, 21831, 18045,   625,   262, 16931,  3290,    13],
        [  464,  2068,  7586, 21831, 18045,   625,   262, 16931,  3290,    13]],
       device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
tensor([10, 10], device='cuda:0')
